In [1]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read Category:Japanese Male Idol (日本の男性アイドル）articles data downloaded from PetScan (https://petscan.wmflabs.org/?psid=22392792)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)
-  read download.csv (日本の男性アイドル(Japanese male idol|1　birth year|3　people|10)

In [2]:
df = pd.read_csv("japaneseMaleIdol.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,稲垣吾郎,4521,NaN,91810,20220701070837
1,2,森且行,4529,NaN,32981,20220619024827
2,3,野口五郎,42949,NaN,69368,20220628065301
3,4,沢田研二,48387,NaN,100419,20220701022606
4,5,水野晴郎,70852,NaN,32353,20220520125617
5,6,勝俣州和,75205,NaN,22817,20220607130457
6,7,郷ひろみ,77880,NaN,170099,20220629131045
7,8,沖田浩之,79456,NaN,35770,20220616075016
8,9,田原俊彦,84182,NaN,67979,20220702001232
9,10,国分太一,86855,NaN,39748,20220610012427


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     193 non-null    int64  
 1   title      193 non-null    object 
 2   pageid     193 non-null    int64  
 3   namespace  0 non-null      float64
 4   length     193 non-null    int64  
 5   touched    193 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 9.2+ KB


In [6]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info


In [8]:
# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 

In [9]:
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [10]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return

In [11]:
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

In [12]:
# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [13]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [14]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [15]:
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

In [22]:
# NEED TO CHANGE for differendt data
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/japaneseMaleIdol.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/japaneseMaleIdol/'

In [23]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [24]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [25]:
not_found = {}      # dict for not-found data

In [26]:
# measure time
start_time = time.time()

In [27]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
1 稲垣吾郎
processing_time(sec):  0.9472804069519043
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
2 森且行
processing_time(sec):  1.1875741481781006
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
3 野口五郎
processing_time(sec):  1.267582893371582
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
4 沢田研二
processing_time(sec):  1.5578856468200684
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
5 水野晴郎
processing_time(sec):  2.691596746444702
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
6 勝俣州和
processing_time(sec):  2.769759178161621
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
7 郷ひろみ
processing_time(sec):  2.9601356983184814
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
8 沖田浩之
processing_time(sec):  3.1170272827148438
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
9 田原俊彦
processing_time(sec):  3.469019889831543
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
10 国

79 渋谷哲平
processing_time(sec):  44.76788926124573
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
80 山本陽一
processing_time(sec):  45.661423206329346
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
81 長田栄二
processing_time(sec):  46.7185583114624
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
82 渡辺徹 (俳優)
processing_time(sec):  47.912304401397705
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
83 あいざき進也
processing_time(sec):  49.141124963760376
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
84 遠藤直人
processing_time(sec):  49.221176862716675
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
85 高木延秀
processing_time(sec):  49.28864026069641
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
86 井上ヨシマサ
processing_time(sec):  49.60454702377319
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
87 長岡尚彦
processing_time(sec):  51.07754993438721
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
88 古川栄司
processing_time(sec):  51.4943523406

157 原嘉孝
processing_time(sec):  113.9942479133606
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
158 成瀬敦志
processing_time(sec):  115.06530332565308
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
159 佐野文哉
processing_time(sec):  115.17491030693054
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
160 中川勝就
processing_time(sec):  115.26155471801758
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
161 浦野秀太
processing_time(sec):  115.34637546539307
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
162 近田力丸
processing_time(sec):  115.74168181419373
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
163 宇野賛多
processing_time(sec):  116.17240190505981
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
164 田淵累生
processing_time(sec):  116.67157745361328
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
165 IGU (B2takes!!)
processing_time(sec):  117.28590559959412
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
166 ゆっきゅん
processing_time(se